In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [1]:
PATH='../../../data/nietzsche/'

In [5]:
get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt',encoding='utf-8').read()
print('corpus length:', len(text))

corpus length: 600893


In [9]:
text[:400]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself '

In [10]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
vocab_size

85

In [11]:
# null character for padding
chars.insert(0,"\0")

In [13]:
' '.join(chars)

'\x00 \n   ! " \' ( ) , - . 0 1 2 3 4 5 6 7 8 9 : ; = ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ ] _ a b c d e f g h i j k l m n o p q r s t u v w x y z Æ ä æ é ë'

In [15]:
char_indicies = dict((c,i) for i,c in enumerate(chars))
indicies_char = dict((i,c) for i,c in enumerate(chars))

In [16]:
idx = [char_indicies[c] for c in text]

In [17]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [19]:
''.join(indicies_char[i] for i in idx[:50])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what th'

### create inputs

In [21]:
cs = 3
c1_dat = [idx[i] for i in range(0,len(idx)-1-cs,cs)]
c2_dat = [idx[i+1] for i in range(0,len(idx)-1-cs,cs)]
c3_dat = [idx[i+2] for i in range(0,len(idx)-1-cs,cs)]
c4_dat = [idx[i+3] for i in range(0,len(idx)-1-cs,cs)]

In [22]:
c1_dat[:10]

[40, 30, 29, 1, 40, 43, 31, 61, 2, 74]

In [24]:
c4_dat[:10]

[30, 29, 1, 40, 43, 31, 61, 2, 74, 2]

In [31]:
#create inputs
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])
#create output
y = np.stack(c4_dat[:-2])

In [32]:
y.shape

(200295,)

### create and train model

In [34]:
n_hidden_act = 256

In [37]:
n_fac = 42

In [36]:
class Char3Model(nn.Module):
    def __init__(self,vocab_size,n_fac,n_hidden_act):
        super().__init__()
        self.e = nn.Embedding(vocab_size,n_fac)
        #layer from input to hidden
        self.l_in = nn.Linear(n_fac,n_hidden_act)
        #layer from hidden to hidden
        self.l_hidden = nn.Linear(n_hidden_act,n_hidden_act)
        #layer from hidden to out
        self.l_out = nn.Linear(n_hidden_act,vocab_size)
    
    def forward(self, c1, c2, c3):
        in1 = F.relu(self.l_in(self.e(c1)))
        in2 = F.relu(self.l_in(self.e(c2)))
        in3 = F.relu(self.l_in(self.e(c3)))
        
        h = V(torch.zeros(in1.size()).cuda())
        h = F.tanh(self.l_hidden(h+in1))
        h = F.tanh(self.l_hidden(h+in2))
        h = F.tanh(self.l_hidden(h+in3))
        
        return F.log_softmax(self.l_out(h))

In [38]:
md = ColumnarModelData.from_arrays(PATH,[-1],np.stack([x1,x2,x3],axis=1),y,bs=512)

In [40]:
m = Char3Model(vocab_size,n_fac,n_hidden_act).cuda()

it = iter(md.trn_dl)
*xs,yt = next(it)
t=m(*V(xs))

t

In [45]:
opt = optim.Adam(m.parameters(),1e-2)

In [47]:
fit(m,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss                               
    0      2.084815   5.615732  



[array([5.61573])]

In [48]:
set_lrs(opt,0.001)

In [49]:
fit(m,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.829241   4.979286  



[array([4.97929])]

### test model